In [2]:
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    MeshcatVisualizer,
    Parser,
    Simulator,
    InverseDynamicsController,
    LeafSystem,
    JointSliders,
    Meshcat,
    FirstOrderLowPassFilter,
    VectorLogSink,
    AbstractValue,
    RigidTransform,
    PublishEvent, 
    TriggerType
)
import numpy as np
from manipulation.utils import RenderDiagram
from robot import Robot
from dynamixel import Dynamixel
import time
import os


In [3]:
class TruncateVec(LeafSystem):
    def __init__(self, input_dims, output_dims):
        LeafSystem.__init__(self)
        self.DeclareVectorInputPort("input", input_dims)
        self.DeclareVectorOutputPort("output", output_dims, self.CalcOutput)
        self.input_dims = input_dims
        self.output_dims = output_dims

    def CalcOutput(self, context, output):
        full_state = self.get_input_port().Eval(context)
        output.SetFromVector(full_state[:self.output_dims]) # TODO: support arbitrary slicing

class ZeroExtendVec(LeafSystem):
    def __init__(self, input_dims, output_dims):
        LeafSystem.__init__(self)
        self.input_dims = input_dims
        self.output_dims = output_dims
        self.DeclareVectorInputPort("input", input_dims)
        self.DeclareVectorOutputPort("output", output_dims, self.CalcOutput)

    def CalcOutput(self, context, output):
        input_vec = self.get_input_port(0).Eval(context)
        output.SetFromVector(np.hstack((input_vec, [0] * (self.output_dims - self.input_dims))))


class Identity(TruncateVec):
    def __init__(self, dims):
        TruncateVec.__init__(self, dims, dims)
        

In [15]:
class SafetyController(LeafSystem):
    def __init__(self, position_dims, body_index, plant ):
        LeafSystem.__init__(self)
        self.DeclareVectorInputPort("desired_pos", position_dims)
        self.DeclareVectorOutputPort("safe_desired_pos", position_dims, self.CalcOutput)
        self._position_dims = position_dims
        self._body_index = body_index
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()

    def CalcOutput(self, context, output):
        desired_pos = self.get_input_port(0).Eval(context)
        # measured_pos = self.get_input_port(1).Eval(context)
        
        self._plant.SetPositions(self._plant_context, desired_pos)
        plant_pose = self._plant.get_body_poses_output_port().Eval(self._plant_context)

        # safety_bounds = np.array([self._safety_bound] * self._position_dims)
        # diff = np.abs(desired_pos - measured_pos)
        # if np.any(diff > safety_bounds):
        #     print(f"Motion too far {diff}")
        #     output.SetFromVector(measured_pos)
        #     return

        # TODO: check FK safety on every joint and/or do intersection with all obstacles here.
        gripper_pose = plant_pose[self._body_index]
        (x,y,z) = gripper_pose.translation()
        # print(f"plant_pose={len(plant_pose)}, z={z}")
        if z < 0.03: # 3cm
            print(f"Too close to ground z={z}")
            # TODO: find a safe pos that's higher up
            output.SetFromVector(desired_pos)
            return
        
        output.SetFromVector(desired_pos)
        
        

In [16]:
def build_sim_robot(meshcat):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-2)
    # Note that we parse into both the plant and the scene_graph here.
    robot_model = Parser(plant, scene_graph).AddModelsFromUrl(
            f"file://{os.getcwd()}/cad/robot-post-processed.urdf"
    )[0]
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("link_0"))
    plant.Finalize()
    
    low_cost_robot = plant.GetModelInstanceByName("onshape")
        
    kp = [100] * plant.num_positions()
    ki = [1] * plant.num_positions()
    kd = [10] * plant.num_positions()
    static_controller = builder.AddSystem(InverseDynamicsController(plant, kp, ki, kd, False))
    extend_pos = builder.AddSystem(ZeroExtendVec(6, 12))
    truncate_state = builder.AddSystem(TruncateVec(12, 6))

    builder.ExportInput(extend_pos.get_input_port(), "desired_position")
    builder.ExportOutput(truncate_state.get_output_port(), "measured_position")
    # builder.ExportOutput(plant.get_body_poses_output_port(), "plant_body_poses")

    # Connect the robot state to the ID controller
    builder.Connect(
        plant.get_state_output_port(low_cost_robot),
        static_controller.get_input_port_estimated_state(),
    )

    # Connect the ID output to the plant
    builder.Connect(
        static_controller.get_output_port_control(), 
        plant.get_actuation_input_port()
    )

    builder.Connect(
        extend_pos.get_output_port(),
        static_controller.get_input_port(1)
    )

    builder.Connect(
        plant.get_state_output_port(low_cost_robot),
        truncate_state.get_input_port()
    )

    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    diagram.set_name("low-cost-robot (sim)")
    return diagram, plant


In [17]:
class LowCostRobot(LeafSystem):
    def __init__(self, plant, num_movable_joints=6):
        LeafSystem.__init__(self)
        dynamixel = Dynamixel.Config(
            baudrate=57_600,
            device_name='/dev/tty.usbmodem578E0211421'
        ).instantiate()
        self._robot = Robot(dynamixel)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._plant_robot = plant.GetModelInstanceByName("onshape")
        # self._robot._disable_torque()
        pos = self._robot.read_position()
        if not pos:
            raise Exception("Could not connect to bot")

        # Slowly move the robot to the home position
        self._home_position = [2048, 1024, 2048, 1709, 2067, 2367]
        self._num_movable_joints = 6
        self._robot.set_goal_pos(self._home_position)

        self.DeclareVectorInputPort("desired_position", 6)
        self.DeclareVectorOutputPort("measured_position", 6, self.CalcOutput)

        # #Declare a periodic event that updates at 10Hz
        # self.DeclarePeriodicEvent(period_sec =1.0/10,
        #                          offset_sec=0.00,
        #                          event=PublishEvent(
        #                              trigger_type=TriggerType.kPeriodic,
        #                              callback=self.StepToPos))

        
        # # NOTE: we need the all_state_ticket because Drake will assume that our output depends on our input, but our output is just state. Tell Drake this by specifying the all_state_ticket since it cannot statically analyze the output.
        # # https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_leaf_system.html#DeclareLeafOutputPort_feedthrough
        # self.DeclareVectorOutputPort("measured_position", 6, self.GetOutput, prerequisites_of_calc={LeafSystem.all_state_ticket()})

    def _slow_move_to_pos(self, desired_position):
        current_position = self._robot.read_position()
        # we want to move a maximum of +100 or -100 in each step
        diff_to_home = [desired_position[i] - current_position[i] for i in range(6)]
        max_diff = max([abs(diff) for diff in diff_to_home])
        time_window = 1
        time_per_step = 0.1
        steps = int(time_window / time_per_step)
        for i in range(steps):
            for j in range(6):
                current_position[j] += int(diff_to_home[j] / steps)
            self._robot.set_goal_pos(current_position)
            time.sleep(time_per_step)
        

    def CalcOutput(self, context, event):
        # should i be setting the plant pos here? hmmm -- no because teh pinv controller does that to calculate the jacobian. (future raghav) eep yes, don't do it in the jacobian why!!
        # first sanitize the input port. just constrain everything from pi/ 3 to -pi / 3.
        # eventually constrain based on fk result
        # next convert [-pi, pi] to [0, 4096]
        # desired position shouldn't be too far from current position (no difference of greater than 100)

        desired_position_float: np.ndarray = self.get_input_port().Eval(context) # type: ignore

        # desired_position = np.clip(desired_position_float, -np.pi / 3, np.pi / 3)
        # TODO: add safety here for joint limits. ideally compute fk and then make sure we aren't hitting below z=0
        desired_position = (desired_position_float + np.pi) / (2 * np.pi) * 4096
        desired_position = desired_position.astype(int)

        measured_position = self._robot.read_position()
        current_pos = np.array(self._get_measured_pos_as_angles())

        # print(context)
        # robot_context = self.(context)
        # self._plant.SetPositions(robot_context, self._plant_robot, current_pos)
        
        # import ipdb; ipdb.set_trace()
        should_move = False
        joint_safety_diffs = [
            # (max_diff, min_diff)
            (300, 25),
            (300, 25),
            (400, 100), # this motor is a bit sticky, so require a greater min
            (300, 25),
            (300, 25),
            (300, 25),
        ]
        for i in range(self._num_movable_joints):
            abs_diff = abs(measured_position[i] - desired_position[i])
            max_diff, min_diff = joint_safety_diffs[i]
            if abs_diff > max_diff:
                print("Desired position too far from current position, skipping")
                print(f"Current pos {measured_position}, desired pos as float {desired_position_float}, desired_position {desired_position}")
                outputs.SetFromVector(current_pos)
                return
                
            elif abs_diff > min_diff:
                # only command a new position if there is somewhere new to move to
                should_move = True

        if should_move:
            print(f"Current pos {measured_position}, desired pos as float {desired_position_float}, desired_position {desired_position}")
            # you can't reuse the measured position list for the rest of the joints because it's an unstable system and will droop!!
            commanded_position = list(desired_position[:self._num_movable_joints]) + list(self._home_position[self._num_movable_joints:])
            print(commanded_position)
            self._robot.set_goal_pos(commanded_position) # gripper is always open for now.

        position = np.array(self._get_measured_pos_as_angles())
        outputs.SetFromVector(position)

    def _get_measured_pos_as_angles(self):
        measured_position = self._robot.read_position()
        return np.array(measured_position) / 4096 * 2 * np.pi - np.pi

    def test_conn(self):
        pos = self.robot.read_position()
        if not pos:
            raise Exception("Could not connect to bot")

    @property
    def robot(self):
        return self._robot


real_robot = None

def build_real_robot(meshcat):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-2)
    # Note that we parse into both the plant and the scene_graph here.
    robot_model = Parser(plant, scene_graph).AddModelsFromUrl(
            f"file://{os.getcwd()}/cad/robot-post-processed.urdf"
    )[0]
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("link_0"))
    plant.Finalize()

    

    # Simulate the robot in the scenegraph so that we get a good visual. 
    # Ideally I would have the plant not have any dynamics config, 
    # and instead just set the plant positions at every time step somehow
    kp = [100] * plant.num_positions()
    ki = [1] * plant.num_positions()
    kd = [10] * plant.num_positions()
    static_controller = builder.AddSystem(InverseDynamicsController(plant, kp, ki, kd, False))
    extend_pos = builder.AddSystem(ZeroExtendVec(6, 12))
    low_cost_robot = plant.GetModelInstanceByName("onshape")
    
    
    builder.Connect(
        extend_pos.get_output_port(),
        static_controller.get_input_port(1)
    )
    builder.Connect(
        plant.get_state_output_port(low_cost_robot),
        static_controller.get_input_port_estimated_state()
    )
    
    builder.Connect(
        static_controller.get_output_port_control(), 
        plant.get_actuation_input_port()
    )
    
    
    # End simulate

    
    # build the leaf system that is the robot
    global real_robot
    real_robot = builder.AddSystem(LowCostRobot(plant))

    builder.Connect(
        real_robot.get_output_port(),
        extend_pos.get_input_port()
    )
    
    builder.ExportInput(real_robot.get_input_port(), "desired_position")
    builder.ExportOutput(real_robot.get_output_port(), "measured_position", )
    
    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    diagram = builder.Build()
    diagram.set_name("low-cost-robot (real)")
    return diagram, plant

In [18]:
def teleop_example(meshcat, real_robot=False):
    if real_robot:
        (robot_diagram, plant) = build_real_robot(meshcat)
    else:
        (robot_diagram, plant) = build_sim_robot(meshcat)
    
    num_joints = 6
    gripper_index = 7
    builder = DiagramBuilder()
    teleop = builder.AddSystem(JointSliders(meshcat=meshcat, plant=plant))
    robot_diagram = builder.AddSystem(robot_diagram)
    safety_controller = builder.AddSystem(SafetyController(num_joints, gripper_index, plant))

    
    # wire up the safety controller
    builder.Connect(teleop.get_output_port(0), safety_controller.get_input_port(0)) # Joint sliders -> input port of safety controller
    builder.Connect(robot_diagram.get_output_port(0), safety_controller.get_input_port(1))
    builder.Connect(safety_controller.get_output_port(0), robot_diagram.get_input_port(0))

    # RenderDiagram(robot_diagram, max_depth=1)

    diagram = builder.Build()
    
    RenderDiagram(diagram, max_depth=2)
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()

    

    robot_context = robot_diagram.GetMyMutableContextFromRoot(context)
    robot_context.SetDiscreteState(np.array([0,-0.67,1.32,0,0,1.24,0,0,0,0,0,0]))
    simulator.AdvanceTo(1e-6)

    initial_plant_pos = robot_diagram.get_output_port(0).Eval(robot_context)
    teleop.SetPositions(initial_plant_pos)
    
    simulator.set_publish_every_time_step(False)
    simulator.set_target_realtime_rate(1.0)
    simulator.AdvanceTo(np.inf)



In [19]:
try:
    meshcat
except NameError:
    meshcat = Meshcat(port=7002)

meshcat.Delete()
meshcat.DeleteAddedControls()

teleop_example(meshcat, real_robot=True)

disabling torque for servos [9, 1, 2, 3, 4, 5]
enabling torque for servos [9, 1, 2, 3, 4, 5]


IndexError: System::get_input_port(): there is no input port with index 1 because there are only 1 input ports in system ::__main__.SafetyController@000000014f6add20.

# Scratch

In [105]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-2)
# Note that we parse into both the plant and the scene_graph here.
robot_model = Parser(plant, scene_graph).AddModelsFromUrl(
        f"file://{os.getcwd()}/cad/robot-post-processed.urdf"
)[0]
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("link_0"))
plant.Finalize()
diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(context)
plant_context.SetDiscreteState(np.zeros(12))
plant.SetPositions(plant_context, [1, 1, 1, 1, 1, 1])
frame = plant.GetFrameByName("moving_side")
frame.CalcPoseInWorld(plant_context)
plant.CreateDefaultContext()
plant.get_body_poses_output_port().Eval(plant_context)

[RigidTransform(
   R=RotationMatrix([
     [1.0, 0.0, 0.0],
     [0.0, 1.0, 0.0],
     [0.0, 0.0, 1.0],
   ]),
   p=[0.0, 0.0, 0.0],
 ),
 RigidTransform(
   R=RotationMatrix([
     [1.0, 0.0, 0.0],
     [0.0, 1.0, 0.0],
     [0.0, 0.0, 1.0],
   ]),
   p=[0.0, 0.0, 0.0],
 ),
 RigidTransform(
   R=RotationMatrix([
     [0.5403023058681377, -0.8414709848078978, -1.0303861190088323e-28],
     [-0.8414709848078978, -0.5403023058681377, 1.2246467991473532e-16],
     [-1.03050474812092e-16, -6.61679489452482e-17, -1.0],
   ]),
   p=[-6.26572873443466e-05, 0.041609087822019435, 0.03908206293678284],
 ),
 RigidTransform(
   R=RotationMatrix([
     [0.44737894712854953, -0.7126889194730921, -0.5403023058681379],
     [0.287258718475728, -0.45761229265187936, 0.8414709848078977],
     [-0.846956023844115, -0.5316629511957431, 1.2253635142879014e-14],
   ]),
   p=[0.008520485973767195, 0.025323519746351327, 0.05523206293678472],
 ),
 RigidTransform(
   R=RotationMatrix([
     [-0.0178026594546349

In [76]:
# real_robot.robot.set_goal_pos([1365, 1329, 1459, 1120, 1249, 1992])